In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# 训练参数设置
kernel_size = 5
batch_size = 64
epoch_num = 10

# 下载MNIST数据集，并加载
train_dataset = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|█████████████████████████████████████████████████████████████████████▉| 9912320/9912422 [32:15<00:00, 6503.27it/s]

Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz



0it [00:00, ?it/s]
  0%|                                                                                        | 0/28881 [00:01<?, ?it/s]
9920512it [32:30, 6503.27it/s]                                                                                         
 85%|██████████████████████████████████████████████████████████████           | 24576/28881 [00:21<00:00, 10535.34it/s]
32768it [00:27, 1080.56it/s]                                                                                           

Extracting ./data/MNIST\raw\train-labels-idx1-ubyte.gz




0it [00:00, ?it/s]

  0%|                                                                                      | 0/1648877 [00:02<?, ?it/s]

  0%|▎                                                                       | 8192/1648877 [00:03<02:31, 10799.34it/s]

  1%|▋                                                                      | 16384/1648877 [00:04<02:03, 13173.77it/s]

  1%|█                                                                      | 24576/1648877 [00:05<02:38, 10233.85it/s]

  2%|█▍                                                                      | 32768/1648877 [00:07<04:23, 6134.52it/s]

  2%|█▊                                                                      | 40960/1648877 [00:08<03:46, 7086.21it/s]

  3%|██▌                                                                     | 57344/1648877 [00:09<02:51, 9277.50it/s]

  4%|██▊                                                                    | 65536/1648877 [00:09<02:13, 11835.46it/s]

  4%|███▏ 

...

 70%|████████████████████████████████████████████████▋                     | 1146880/1648877 [02:49<02:36, 3208.69it/s]

 70%|█████████████████████████████████████████████████                     | 1155072/1648877 [02:49<02:01, 4078.87it/s]

 71%|█████████████████████████████████████████████████▍                    | 1163264/1648877 [02:51<01:44, 4646.66it/s]

 71%|█████████████████████████████████████████████████▋                    | 1171456/1648877 [02:52<01:32, 5142.57it/s]

 72%|██████████████████████████████████████████████████                    | 1179648/1648877 [02:52<01:08, 6890.72it/s]

 72%|██████████████████████████████████████████████████▍                   | 1187840/1648877 [02:54<01:12, 6332.51it/s]

 73%|██████████████████████████████████████████████████▊                   | 1196032/1648877 [02:56<01:36, 4671.92it/s]

 73%|██████████████████████████████████████████████████▊                   | 1196032/1648877 [03:14<01:36, 4671.92it/s]

 73%|███████████████████████████

Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz





0it [00:00, ?it/s]


  0%|                                                                                         | 0/4542 [00:01<?, ?it/s]


8192it [00:01, 5392.56it/s]                                                                                            

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!




1654784it [04:34, 9120.35it/s]

In [3]:
# 网络搭建
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=kernel_size)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = func.relu(self.mp(self.conv1(x)))
        x = func.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return func.log_softmax(x)


In [4]:
# 生成实例，选择优化器
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [5]:
# 训练过程
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = func.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [6]:
# 测试过程
def test():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += func.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [7]:
# 主函数
if __name__=="__main__":
    for epoch in range(1, epoch_num):
        train(epoch)
        test()

D:\ruanjian\Anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.282939
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.738884
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.278553
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.193705
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.280747


D:\ruanjian\Anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
D:\ruanjian\Anaconda\anaconda\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1846, Accuracy: 9471/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.146543
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.275967
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.098828
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.118345
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.230747

Test set: Average loss: 0.1182, Accuracy: 9651/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.122863
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.236877
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.053131
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.062733
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.134989

Test set: Average loss: 0.0876, Accuracy: 9711/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.070573
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.066077
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.149758
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.131723
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.074675

Test set: Average loss: 0.0787, Accuracy: 9737/10000 (97%)

Train Epoch: 5 [0/600